API documentation

https://the-odds-api.com/liveapi/guides/v4/#more-info-3


need to get the Event ID 

Need to pass the invent IDs interavtive to the player props function

Extract all Event ID for specified Date Range

In [88]:
import requests
import pandas as pd

def get_historical_events(api_key, sport, date, date_format='iso'):
    params = {
        'apiKey': api_key,
        'date': date,
        'dateFormat': date_format
    }
    
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{sport}/events', params=params)
    
    if response.status_code == 200:
        return response.json()  # Return the JSON data for further processing
    else:
        print(f'Failed to get events for date {date}: Status code {response.status_code}, Response body {response.text}')
        return []  # Return an empty list in case of an HTTP error

API_KEY = 'e831e5b2b5b2a411ccc6196f0ccaf379'
SPORT = 'basketball_nba'

# Read the dates CSV
event_dates_df = pd.read_csv('Event Dates V1.csv')
event_dates_df['Dates'] = pd.to_datetime(event_dates_df['Dates'], format='%m/%d/%y')     #API requires this date time format!!!!
event_dates_df['Dates_iso'] = event_dates_df['Dates'].dt.strftime('%Y-%m-%dT05:30:00Z')
event_dates_df
# Iterate over the dates in the DataFrame
all_events = []
for date in event_dates_df['Dates_iso']:
    daily_event_data = get_historical_events(API_KEY, SPORT, date)
    # Check if 'data' key exists in the response and it contains a list of events
    if 'data' in daily_event_data and isinstance(daily_event_data['data'], list):
        all_events.extend(daily_event_data['data'])  # Append the list of events

# Convert the list of all events to a DataFrame
all_events_df = pd.DataFrame(all_events)
all_events_df



Failed to get events for date 2023-05-03T05:30:00Z: Status code 401, Response body {"message":"Request quota has been reached. See usage plans at https://the-odds-api.com"}

Failed to get events for date 2023-05-04T05:30:00Z: Status code 401, Response body {"message":"Request quota has been reached. See usage plans at https://the-odds-api.com"}

Failed to get events for date 2023-05-05T05:30:00Z: Status code 401, Response body {"message":"Request quota has been reached. See usage plans at https://the-odds-api.com"}

Failed to get events for date 2023-05-06T05:30:00Z: Status code 401, Response body {"message":"Request quota has been reached. See usage plans at https://the-odds-api.com"}

Failed to get events for date 2023-05-07T05:30:00Z: Status code 401, Response body {"message":"Request quota has been reached. See usage plans at https://the-odds-api.com"}

Failed to get events for date 2023-05-08T05:30:00Z: Status code 401, Response body {"message":"Request quota has been reached. See

KeyboardInterrupt: 

In [80]:
# Save the DataFrame to a CSV file
all_events_df.to_csv('/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/complete_eventid.csv', index=False)  # Replace with your desired output path

Event ids will be the an iteration item that is used to pull the player props for that specific event 
Ex: Event ID: f68297ad59b106aaf5970a0a385f539f will allow us to pull player props for BOS vs. Phil

In [75]:
# #trouble shooting data type issue

# # Check the structure of the first response to understand how to extract the data
# test_date = event_dates_df['Dates_iso'][0]
# test_response = get_historical_events(API_KEY, SPORT, test_date)

# print(type(test_response))  # This will tell us whether it's a list or a dictionary
# print(test_response)        # This will show us the contents so we know how to access the data


<class 'dict'>
{'timestamp': '2023-05-03T05:25:40Z', 'previous_timestamp': '2023-05-03T05:20:39Z', 'next_timestamp': '2023-05-03T05:30:39Z', 'data': [{'id': 'f68297ad59b106aaf5970a0a385f539f', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-05-04T00:00:00Z', 'home_team': 'Boston Celtics', 'away_team': 'Philadelphia 76ers'}, {'id': 'fca32c7f3e4a6797ee8a56815ebc142c', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-05-05T01:00:00Z', 'home_team': 'Golden State Warriors', 'away_team': 'Los Angeles Lakers'}, {'id': '5d11b12a18a30c985f2903d308d947ef', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-05-06T02:00:00Z', 'home_team': 'Phoenix Suns', 'away_team': 'Denver Nuggets'}, {'id': '78c0cc10104266a5ef48c1acc91cfa1c', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2023-05-06T19:30:00Z', 'home_team': 'Miami Heat', 'away_team': 'New York Knicks'}]}


API call GET player props ---- [Points]
 Ex: Lebron 20 pts

In [3]:
event_ids_df = pd.read_csv('/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/complete_eventid.csv')
print(event_ids_df['commence_time'].dtype)

object


In [4]:
# Load and prepare event IDs and dates
event_ids_df = pd.read_csv('complete_eventid.csv')
event_ids_df['commence_time'] = pd.to_datetime(event_ids_df['commence_time'])
print(event_ids_df['commence_time'].dtype)

datetime64[ns, UTC]


In [7]:
                                        ### SUCCESSFULLY PULLS THE PLAYER PROPS [POINTS] USING THE EVENT ID, DATE AS ITERATIVE ELEMENTS


import requests
import pandas as pd
from datetime import datetime

def query_historical_event_odds(api_key, sport, event_id, regions, markets, odds_format, date_format, date):
    url = f'https://api.the-odds-api.com/v4/historical/sports/{sport}/events/{event_id}/odds'
    params = {
        'api_key': api_key,
        'regions': regions,
        'markets': markets,
        'oddsFormat': odds_format,
        'dateFormat': date_format,
        'date': date.strftime('%Y-%m-%dT%H:%M:%SZ')
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            return data['data']  # Make sure this is the part that contains the odds
        else:
            print(f"No 'data' key in response for event {event_id}.")
            return None
    else:
        print(f'Failed to get odds for event {event_id}: status code {response.status_code}, response body {response.text}')
        return None

API_KEY = 'f65284c4929f0172f229ebd743b1c436'
SPORT = 'basketball_nba'
REGIONS = 'us'
MARKETS = 'player_points'
ODDS_FORMAT = 'decimal'
DATE_FORMAT = 'iso'

# Loading the CSV file
event_ids_df = pd.read_csv('complete_eventid.csv')
event_ids_df['commence_time'] = pd.to_datetime(event_ids_df['commence_time'])

# Fetching the odds for each event
all_odds_data = []
for _, row in event_ids_df.iterrows():
    event_id = row['id']
    date = row['commence_time']
    odds_data = query_historical_event_odds(API_KEY, SPORT, event_id, REGIONS, MARKETS, ODDS_FORMAT, DATE_FORMAT, date)
    if odds_data:
        all_odds_data.append(odds_data)  # Use append instead of extend if the data structure requires it

# Creating a DataFrame to hold all fetched data
all_odds_df = pd.DataFrame(all_odds_data)
print(all_odds_df.head())


Failed to get odds for event 22daf55e428be9bfc9f0dafd462d893f: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 20fc47de6083f99da2027876dfebadb7: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 9cb979b834e23b9dc8737eebba4db760: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 20fc47de6083f99da2027876dfebadb7: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 5a4b28938902c4fba16da19c6e8c1296: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 9cb979b834e23b9dc8737eebba4db760: status code 404, response body {

In [44]:
#save the to player_props_points.csv

#all_odds_df
all_odds_df.to_csv('eventid_player_props_points.csv')


# requires further breakdown!!!!!!!!!
final_playerprop_df = pd.read_csv('eventid_player_props_points.csv')

All odds dataframe requires breakdown of JSON data within the bookmakers column 

In [7]:
                                                                #JSON breakdown cell -----[POINTS]!!!!

import pandas as pd
import json
import re

def clean_json(json_str):
    # Escape all double quotes within strings
    json_str = json_str.replace(r'\"', '\\"')
    
    # Convert single quotes to double quotes around JSON keys and values
    json_str = re.sub(r"(?<=[:\[{,\s])'([^']*)'(?=[,\]}:\s])", r'"\1"', json_str)
    
    # Remove trailing commas which cause issues in JSON parsing
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*\]', ']', json_str)
    
    return json_str

def expand_bookmakers(modified_playerprop):
    all_rows = []
    
    for index, row in modified_playerprop.iterrows():
        bookmakers_str = row['bookmakers']
        if isinstance(bookmakers_str, str):
            try:
                # Clean and load the JSON string
                cleaned_str = clean_json(bookmakers_str)
                bookmakers = json.loads(cleaned_str)
            except json.JSONDecodeError as e:
                error_snippet = bookmakers_str[max(0, e.pos-50):e.pos+50]
                print(f"Error decoding JSON at row {index}: {e} at ...{error_snippet}...")
                continue  # Skip rows with JSON errors
        else:
            bookmakers = bookmakers_str

        # Normalize bookmaker data
        bookmakers = pd.json_normalize(bookmakers)
        for _, bookmaker in bookmakers.iterrows():
            for market in bookmaker['markets']:
                for outcome in market['outcomes']:
                    # Create a new row for each outcome, including bookmaker and market details
                    new_row = {
                        'id': row['id'],
                        'bookmaker_key': bookmaker['key'],
                        'bookmaker_title': bookmaker['title'],
                        'market_key': market['key'],
                        'outcome': outcome['name'],
                        'player_name': outcome['description'],
                        'price': outcome['price'],
                        'point': outcome['point'],
                    }
                    all_rows.append(new_row)
    
    # Convert the list of dictionaries into a DataFrame
    return pd.DataFrame(all_rows)

# Assuming you have already loaded your DataFrame
try:
    modified_playerprop = pd.read_csv('eventid_player_props_points.csv')
    if 'id' in modified_playerprop.columns:
        points_expanded_df = expand_bookmakers(modified_playerprop)
        print(points_expanded_df.head(3))
    else:
        print("DataFrame does not contain 'id'. Please check your data.")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Empty data error: {e}")


                                 id bookmaker_key bookmaker_title  \
0  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   
1  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   
2  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   

      market_key outcome        player_name  price  point  
0  player_points    Over         Al Horford   1.95    8.5  
1  player_points   Under         Al Horford   1.80    8.5  
2  player_points    Over  De'Anthony Melton   2.25    8.5  


In [8]:
#broken down JSON file of NBA player props bets (POINTS)
points_expanded_df.head(2)

,id,bookmaker_key,bookmaker_title,market_key,outcome,player_name,price,point
0,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_points,Over,Al Horford,1.95,8.5
1,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_points,Under,Al Horford,1.80,8.5


Player prop data extraction - [Rebounds] using event ID

In [47]:
                                        ### SUCCESSFULLY PULLS THE PLAYER PROPS [REBOUNDS] USING THE EVENT ID, DATE AS ITERATIVE ELEMENTS


import requests
import pandas as pd
from datetime import datetime

def query_historical_event_odds(api_key, sport, event_id, regions, markets, odds_format, date_format, date):
    url = f'https://api.the-odds-api.com/v4/historical/sports/{sport}/events/{event_id}/odds'
    params = {
        'api_key': api_key,
        'regions': regions,
        'markets': markets,
        'oddsFormat': odds_format,
        'dateFormat': date_format,
        'date': date.strftime('%Y-%m-%dT%H:%M:%SZ')
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            return data['data']  # Make sure this is the part that contains the odds
        else:
            print(f"No 'data' key in response for event {event_id}.")
            return None
    else:
        print(f'Failed to get odds for event {event_id}: status code {response.status_code}, response body {response.text}')
        return None

API_KEY = 'f65284c4929f0172f229ebd743b1c436'
SPORT = 'basketball_nba'
REGIONS = 'us'
MARKETS = 'player_rebounds'
ODDS_FORMAT = 'decimal'
DATE_FORMAT = 'iso'

# Loading the CSV file
event_ids_df = pd.read_csv('/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/complete_eventid.csv')
event_ids_df['commence_time'] = pd.to_datetime(event_ids_df['commence_time'])

# Fetching the odds for each event
all_odds_data = []
for _, row in event_ids_df.iterrows():
    event_id = row['id']
    date = row['commence_time']
    odds_data = query_historical_event_odds(API_KEY, SPORT, event_id, REGIONS, MARKETS, ODDS_FORMAT, DATE_FORMAT, date)
    if odds_data:
        all_odds_data.append(odds_data)  # Use append instead of extend if the data structure requires it

# Creating a DataFrame to hold all fetched data
player_rebounds_df = pd.DataFrame(all_odds_data)
player_rebounds_df

player_rebounds_df.to_csv('/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/eventid_player_props_rebounds.csv')




Failed to get odds for event 22daf55e428be9bfc9f0dafd462d893f: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 20fc47de6083f99da2027876dfebadb7: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 9cb979b834e23b9dc8737eebba4db760: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 20fc47de6083f99da2027876dfebadb7: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 5a4b28938902c4fba16da19c6e8c1296: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 9cb979b834e23b9dc8737eebba4db760: status code 404, response body {

In [5]:
                                                                #JSON breakdown cell   [REBOUNDS]!!!!

import pandas as pd
import json
import re

def clean_json(json_str):
    # Escape all double quotes within strings
    json_str = json_str.replace(r'\"', '\\"')
    
    # Convert single quotes to double quotes around JSON keys and values
    json_str = re.sub(r"(?<=[:\[{,\s])'([^']*)'(?=[,\]}:\s])", r'"\1"', json_str)
    
    # Remove trailing commas which cause issues in JSON parsing
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*\]', ']', json_str)
    
    return json_str

def expand_bookmakers(modified_playerprop):
    all_rows = []
    
    for index, row in modified_playerprop.iterrows():
        bookmakers_str = row['bookmakers']
        if isinstance(bookmakers_str, str):
            try:
                # Clean and load the JSON string
                cleaned_str = clean_json(bookmakers_str)
                bookmakers = json.loads(cleaned_str)
            except json.JSONDecodeError as e:
                error_snippet = bookmakers_str[max(0, e.pos-50):e.pos+50]
                print(f"Error decoding JSON at row {index}: {e} at ...{error_snippet}...")
                continue  # Skip rows with JSON errors
        else:
            bookmakers = bookmakers_str

        # Normalize bookmaker data
        bookmakers = pd.json_normalize(bookmakers)
        for _, bookmaker in bookmakers.iterrows():
            for market in bookmaker['markets']:
                for outcome in market['outcomes']:
                    # Create a new row for each outcome, including bookmaker and market details
                    new_row = {
                        'id': row['id'],
                        'bookmaker_key': bookmaker['key'],
                        'bookmaker_title': bookmaker['title'],
                        'market_key': market['key'],
                        'outcome': outcome['name'],
                        'player_name': outcome['description'],
                        'price': outcome['price'],
                        'rebounds': outcome['point'],
                    }
                    all_rows.append(new_row)
    
    # Convert the list of dictionaries into a DataFrame
    return pd.DataFrame(all_rows)

# Assuming you have already loaded your DataFrame
try:
    modified_playerprop = pd.read_csv('/Users/byrondowdell/Library/CloudStorage/OneDrive-Personal/Grad School/Spring 2024/Advanced Machine Learning/Project/Sports Betting/eventid_player_props_rebounds.csv')
    if 'id' in modified_playerprop.columns:
        rebounds_expanded_df = expand_bookmakers(modified_playerprop)
        print(rebounds_expanded_df.head(3))
    else:
        print("DataFrame does not contain 'id'. Please check your data.")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Empty data error: {e}")


                                 id bookmaker_key bookmaker_title  \
0  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   
1  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   
2  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   

        market_key outcome        player_name  price  rebounds  
0  player_rebounds    Over         Al Horford   1.65       5.5  
1  player_rebounds   Under         Al Horford   2.25       5.5  
2  player_rebounds    Over  De'Anthony Melton   1.95       2.5  


In [9]:
rebounds_expanded_df.head(2)

,id,bookmaker_key,bookmaker_title,market_key,outcome,player_name,price,rebounds
0,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_rebounds,Over,Al Horford,1.65,5.5
1,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_rebounds,Under,Al Horford,2.25,5.5


In [59]:
                                        ### SUCCESSFULLY PULLS THE PLAYER PROPS [Assists] USING THE EVENT ID, DATE AS ITERATIVE ELEMENTS


import requests
import pandas as pd
from datetime import datetime

def query_historical_event_odds(api_key, sport, event_id, regions, markets, odds_format, date_format, date):
    url = f'https://api.the-odds-api.com/v4/historical/sports/{sport}/events/{event_id}/odds'
    params = {
        'api_key': api_key,
        'regions': regions,
        'markets': markets,
        'oddsFormat': odds_format,
        'dateFormat': date_format,
        'date': date.strftime('%Y-%m-%dT%H:%M:%SZ')
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            return data['data']  # Make sure this is the part that contains the odds
        else:
            print(f"No 'data' key in response for event {event_id}.")
            return None
    else:
        print(f'Failed to get odds for event {event_id}: status code {response.status_code}, response body {response.text}')
        return None

API_KEY = 'f65284c4929f0172f229ebd743b1c436'
SPORT = 'basketball_nba'
REGIONS = 'us'
MARKETS = 'player_assists'
ODDS_FORMAT = 'decimal'
DATE_FORMAT = 'iso'

# Loading the CSV file
event_ids_df = pd.read_csv('complete_eventid.csv')
event_ids_df['commence_time'] = pd.to_datetime(event_ids_df['commence_time'])

# Fetching the odds for each event
all_odds_data = []
for _, row in event_ids_df.iterrows():
    event_id = row['id']
    date = row['commence_time']
    odds_data = query_historical_event_odds(API_KEY, SPORT, event_id, REGIONS, MARKETS, ODDS_FORMAT, DATE_FORMAT, date)
    if odds_data:
        all_odds_data.append(odds_data)  # Use append instead of extend if the data structure requires it

# Creating a DataFrame to hold all fetched data
player_assists_df = pd.DataFrame(all_odds_data)
player_assists_df

player_assists_df.to_csv('eventid_player_props_assist.csv')


Failed to get odds for event 22daf55e428be9bfc9f0dafd462d893f: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 20fc47de6083f99da2027876dfebadb7: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 9cb979b834e23b9dc8737eebba4db760: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 20fc47de6083f99da2027876dfebadb7: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 5a4b28938902c4fba16da19c6e8c1296: status code 404, response body {"message":"Event not found. The event may have expired or the event id is invalid."}

Failed to get odds for event 9cb979b834e23b9dc8737eebba4db760: status code 404, response body {

In [71]:
player_assists_df

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers
0,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:10:00Z,Boston Celtics,Philadelphia 76ers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
1,fca32c7f3e4a6797ee8a56815ebc142c,basketball_nba,NBA,2023-05-05T01:05:00Z,Golden State Warriors,Los Angeles Lakers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
2,5d11b12a18a30c985f2903d308d947ef,basketball_nba,NBA,2023-05-06T02:15:00Z,Phoenix Suns,Denver Nuggets,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
3,78c0cc10104266a5ef48c1acc91cfa1c,basketball_nba,NBA,2023-05-06T19:30:00Z,Miami Heat,New York Knicks,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
4,fca32c7f3e4a6797ee8a56815ebc142c,basketball_nba,NBA,2023-05-05T01:05:00Z,Golden State Warriors,Los Angeles Lakers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
...,...,...,...,...,...,...,...
5075,0fd3e3ee6cadcae1adf05b0b3fcd1692,basketball_nba,NBA,2024-04-20T17:14:33Z,Cleveland Cavaliers,Orlando Magic,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
5076,58fe88d816c30e8186084fd60e8bc1fe,basketball_nba,NBA,2024-04-20T19:30:00Z,Minnesota Timberwolves,Phoenix Suns,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
5077,ef3b75810319700a88a1be226ccc7b50,basketball_nba,NBA,2024-04-21T00:40:00Z,Denver Nuggets,Los Angeles Lakers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."
5078,7611315f3f28327571d9f58212774a99,basketball_nba,NBA,2024-04-21T23:10:00Z,Milwaukee Bucks,Indiana Pacers,"[{'key': 'draftkings', 'title': 'DraftKings', ..."


In [10]:
                                                                #JSON assist breakdown cell!!!!

import pandas as pd
import json
import re

def clean_json(json_str):
    # Escape all double quotes within strings
    json_str = json_str.replace(r'\"', '\\"')
    
    # Convert single quotes to double quotes around JSON keys and values
    json_str = re.sub(r"(?<=[:\[{,\s])'([^']*)'(?=[,\]}:\s])", r'"\1"', json_str)
    
    # Remove trailing commas which cause issues in JSON parsing
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*\]', ']', json_str)
    
    return json_str

def expand_bookmakers(modified_playerprop):
    all_rows = []
    
    for index, row in modified_playerprop.iterrows():
        bookmakers_str = row['bookmakers']
        if isinstance(bookmakers_str, str):
            try:
                # Clean and load the JSON string
                cleaned_str = clean_json(bookmakers_str)
                bookmakers = json.loads(cleaned_str)
            except json.JSONDecodeError as e:
                error_snippet = bookmakers_str[max(0, e.pos-50):e.pos+50]
                print(f"Error decoding JSON at row {index}: {e} at ...{error_snippet}...")
                continue  # Skip rows with JSON errors
        else:
            bookmakers = bookmakers_str

        # Normalize bookmaker data
        bookmakers = pd.json_normalize(bookmakers)
        for _, bookmaker in bookmakers.iterrows():
            for market in bookmaker['markets']:
                for outcome in market['outcomes']:
                    # Create a new row for each outcome, including bookmaker and market details
                    new_row = {
                        'id': row['id'],
                        'bookmaker_key': bookmaker['key'],
                        'bookmaker_title': bookmaker['title'],
                        'market_key': market['key'],
                        'outcome': outcome['name'],
                        'player_name': outcome['description'],
                        'price': outcome['price'],
                        'assists': outcome['point'],
                    }
                    all_rows.append(new_row)
    
    # Convert the list of dictionaries into a DataFrame
    return pd.DataFrame(all_rows)

# Assuming you have already loaded your DataFrame
try:
    modified_playerprop = pd.read_csv('eventid_player_props_assist.csv')
    if 'id' in modified_playerprop.columns:
        assists_expanded_df = expand_bookmakers(modified_playerprop)
        print(assists_expanded_df.head(3))
    else:
        print("DataFrame does not contain 'id'. Please check your data.")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Empty data error: {e}")


                                 id bookmaker_key bookmaker_title  \
0  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   
1  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   
2  f68297ad59b106aaf5970a0a385f539f    draftkings      DraftKings   

       market_key outcome    player_name  price  assists  
0  player_assists    Over     Al Horford   2.35      3.5  
1  player_assists   Under     Al Horford   1.61      3.5  
2  player_assists    Over  Derrick White   2.25      3.5  


union/stack the 3 player prop data frames [points,rebounds and assist] --- Concat

In [11]:
# Rename the last column to 'prop'
assists_expanded_df.columns = [*assists_expanded_df.columns[:-1], 'prop']
rebounds_expanded_df.columns = [*rebounds_expanded_df.columns[:-1], 'prop']
points_expanded_df.columns = [*points_expanded_df.columns[:-1], 'prop']


points_rebounds_assists_df = pd.concat([assists_expanded_df, rebounds_expanded_df, points_expanded_df], axis=0, ignore_index=True)
points_rebounds_assists_df[:-20]


,id,bookmaker_key,bookmaker_title,market_key,outcome,player_name,price,prop
0,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_assists,Over,Al Horford,2.35,3.5
1,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_assists,Under,Al Horford,1.61,3.5
2,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_assists,Over,Derrick White,2.25,3.5
3,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_assists,Under,Derrick White,1.65,3.5
4,f68297ad59b106aaf5970a0a385f539f,draftkings,DraftKings,player_assists,Over,James Harden,1.87,8.5
...,...,...,...,...,...,...,...,...
4923266,7aa3ab404262a55d4c5473372324c52a,betrivers,BetRivers,player_points,Under,Tim Hardaway Jr,1.82,7.5
4923267,7aa3ab404262a55d4c5473372324c52a,betrivers,BetRivers,player_points,Over,Tim Hardaway Jr,1.73,6.5
4923268,7aa3ab404262a55d4c5473372324c52a,betrivers,BetRivers,player_points,Under,Tim Hardaway Jr,2.04,6.5
4923269,7aa3ab404262a55d4c5473372324c52a,betrivers,BetRivers,player_points,Over,Tim Hardaway Jr,1.40,5.5


In [28]:
all_events_df = pd.read_csv('complete_eventid.csv') 

all_events_df.head(2)

,id,sport_key,sport_title,commence_time,home_team,away_team
0,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers
1,fca32c7f3e4a6797ee8a56815ebc142c,basketball_nba,NBA,2023-05-05T01:00:00Z,Golden State Warriors,Los Angeles Lakers


Merge all_event_df with expanded points_rebounds_assists_df ---- to get date and team matchup

In [31]:
##COMPLETE Players prop dataset [points, rebounds, assists]
import pandas as pd

merged_player_points_eventid = pd.merge(all_events_df, points_rebounds_assists_df,  on='id', how='left')


#Rename the column 'market_key' to 'prop_market'
merged_player_points_eventid = merged_player_points_eventid.rename(columns={'market_key': 'prop_market'})

merged_player_points_eventid

,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,prop_market,outcome,player_name,price,prop
0,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Al Horford,2.35,3.5
1,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Al Horford,1.61,3.5
2,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Derrick White,2.25,3.5
3,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Derrick White,1.65,3.5
4,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,James Harden,1.87,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271081944,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Under,Dereck Lively II,1.91,5.5
271081945,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Over,Josh Green,1.87,4.5
271081946,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Under,Josh Green,1.87,4.5
271081947,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Over,Dante Exum,1.87,4.5


In [2]:
#save merged player props
merged_player_points_eventid.to_csv('eventid_playerprop_pts_rebs_asts.csv')

NameError: name 'merged_player_points_eventid' is not defined

In [2]:
import pandas as pd

merged_player_points_eventid = pd.read_csv('eventid_playerprop_pts_rebs_asts.csv')

merged_player_points_eventid

,Unnamed: 0,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,prop_market,outcome,player_name,price,prop
0,0,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Al Horford,2.35,3.5
1,1,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Al Horford,1.61,3.5
2,2,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Derrick White,2.25,3.5
3,3,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Derrick White,1.65,3.5
4,4,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,James Harden,1.87,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271081944,271081944,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Under,Dereck Lively II,1.91,5.5
271081945,271081945,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Over,Josh Green,1.87,4.5
271081946,271081946,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Under,Josh Green,1.87,4.5
271081947,271081947,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Over,Dante Exum,1.87,4.5


In [3]:
#create a MATCHUP column ---- this field will be used to determine the outcome of a the bet
import pandas as pd

# Load the abbreviation data
abbreviations_df = pd.read_csv('NBA_Data/NBA team abbreviations.csv')

# Create a dictionary mapping the full team name to its abbreviation
team_to_abbreviation = dict(zip(abbreviations_df['Team Name'], abbreviations_df['Abbreviations']))

def create_matchup_column(df, team_to_abbreviation):
    # Helper function to apply the mapping
    def get_abbreviation(team_name):
        return team_to_abbreviation[team_name]

    # Apply the mapping to the home and away team columns
    df['MATCHUP'] = df['home_team'].apply(get_abbreviation) + ' vs. ' + df['away_team'].apply(get_abbreviation)
    return df


#apply the function:
updated_merged_player_points_eventid = create_matchup_column(merged_player_points_eventid, team_to_abbreviation)


updated_merged_player_points_eventid



,Unnamed: 0,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,prop_market,outcome,player_name,price,prop,MATCHUP
0,0,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Al Horford,2.35,3.5,BOS vs. PHI
1,1,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Al Horford,1.61,3.5,BOS vs. PHI
2,2,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Derrick White,2.25,3.5,BOS vs. PHI
3,3,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Derrick White,1.65,3.5,BOS vs. PHI
4,4,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,James Harden,1.87,8.5,BOS vs. PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271081944,271081944,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Under,Dereck Lively II,1.91,5.5,LAC vs. DAL
271081945,271081945,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Over,Josh Green,1.87,4.5,LAC vs. DAL
271081946,271081946,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Under,Josh Green,1.87,4.5,LAC vs. DAL
271081947,271081947,7aa3ab404262a55d4c5473372324c52a,basketball_nba,NBA,2024-04-21T21:10:00Z,Los Angeles Clippers,Dallas Mavericks,betmgm,BetMGM,player_points,Over,Dante Exum,1.87,4.5,LAC vs. DAL


Use NBA data from kaggle to merge player file with 2023/24 game file 

In [4]:
import pandas as pd

gamedata_23_24df= pd.read_csv('2023_new.csv')
gamedata_23_24df

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Date_Game
0,22023,101108,22300490,"JAN 05, 2024",GSW vs. DET,W,24,3,5,0.600,...,6,4,0,1,1,2,8,2,1,NaN
1,22023,101108,22300478,"JAN 04, 2024",GSW vs. DEN,L,30,4,7,0.571,...,4,6,0,0,0,2,9,4,1,NaN
2,22023,101108,22300463,"JAN 02, 2024",GSW vs. ORL,W,30,3,5,0.600,...,2,4,0,0,1,1,12,-8,1,NaN
3,22023,101108,22300444,"DEC 30, 2023",GSW vs. DAL,L,33,9,14,0.643,...,2,6,2,1,0,2,24,-10,1,NaN
4,22023,101108,22300426,"DEC 28, 2023",GSW vs. MIA,L,27,3,6,0.500,...,4,4,0,0,3,4,9,-8,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21102,22023,1627826,22300880,"MAR 04, 2024",LAC @ MIL,L,24,2,4,0.500,...,11,4,0,0,2,3,6,3,1,2024-03-04
21103,22023,1627826,22300873,"MAR 03, 2024",LAC @ MIN,W,19,1,2,0.500,...,2,1,0,0,1,0,2,-11,1,2024-03-03
21104,22023,1627826,22300827,"FEB 25, 2024",LAC vs. SAC,L,33,8,9,0.889,...,6,2,1,0,4,3,16,-6,1,2024-02-25
21105,22023,1627826,22300807,"FEB 23, 2024",LAC @ MEM,W,27,5,7,0.714,...,10,2,0,0,0,6,10,6,1,2024-02-23


In [5]:
NBA_players = pd.read_csv('NBA_PLAYERS.csv')

# Filter the DataFrame to include only active players
active_players_df = NBA_players[NBA_players['is_active'] == True]
active_players_df = active_players_df.rename(columns={'id': 'Player_ID'})

# Display the first two rows of the filtered DataFrame
active_players_df.head(2)

,Player_ID,full_name,first_name,last_name,is_active
10,1630173,Precious Achiuwa,Precious,Achiuwa,True
22,203500,Steven Adams,Steven,Adams,True


In [6]:
#merge active players with 23/24 game data
merged_NBA_player_game = pd.merge(active_players_df, gamedata_23_24df,  on='Player_ID', how='left')
merged_NBA_player_game

,Player_ID,full_name,first_name,last_name,is_active,SEASON_ID,Game_ID,GAME_DATE,MATCHUP,WL,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Date_Game
0,1630173,Precious Achiuwa,Precious,Achiuwa,True,22023.0,22300778.0,"FEB 14, 2024",NYK @ ORL,L,...,14.0,5.0,0.0,0.0,1.0,1.0,23.0,-4.0,1.0,NaN
1,1630173,Precious Achiuwa,Precious,Achiuwa,True,22023.0,22300765.0,"FEB 12, 2024",NYK @ HOU,L,...,17.0,0.0,0.0,3.0,3.0,4.0,8.0,3.0,1.0,NaN
2,1630173,Precious Achiuwa,Precious,Achiuwa,True,22023.0,22300755.0,"FEB 10, 2024",NYK vs. IND,L,...,6.0,0.0,2.0,3.0,2.0,2.0,15.0,-12.0,1.0,NaN
3,1630173,Precious Achiuwa,Precious,Achiuwa,True,22023.0,22300736.0,"FEB 08, 2024",NYK vs. DAL,L,...,16.0,3.0,0.0,1.0,3.0,5.0,13.0,-14.0,1.0,NaN
4,1630173,Precious Achiuwa,Precious,Achiuwa,True,22023.0,22300722.0,"FEB 06, 2024",NYK vs. MEM,W,...,5.0,3.0,3.0,1.0,0.0,2.0,17.0,-1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16792,1627826,Ivica Zubac,Ivica,Zubac,True,22023.0,22300880.0,"MAR 04, 2024",LAC @ MIL,L,...,11.0,4.0,0.0,0.0,2.0,3.0,6.0,3.0,1.0,2024-03-04
16793,1627826,Ivica Zubac,Ivica,Zubac,True,22023.0,22300873.0,"MAR 03, 2024",LAC @ MIN,W,...,2.0,1.0,0.0,0.0,1.0,0.0,2.0,-11.0,1.0,2024-03-03
16794,1627826,Ivica Zubac,Ivica,Zubac,True,22023.0,22300827.0,"FEB 25, 2024",LAC vs. SAC,L,...,6.0,2.0,1.0,0.0,4.0,3.0,16.0,-6.0,1.0,2024-02-25
16795,1627826,Ivica Zubac,Ivica,Zubac,True,22023.0,22300807.0,"FEB 23, 2024",LAC @ MEM,W,...,10.0,2.0,0.0,0.0,0.0,6.0,10.0,6.0,1.0,2024-02-23


In [17]:
column_names = merged_NBA_player_game.columns

column_names

Index(['Player_ID', 'full_name', 'first_name', 'last_name', 'is_active',
       'SEASON_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', 'VIDEO_AVAILABLE', 'Date_Game'],
      dtype='object')

In [9]:
# Drop specified columns from the DataFrame
#merged_NBA_player_game = merged_NBA_player_game.drop(columns=['VIDEO_AVAILABLE', 'Date_Game', 'is_active', 'first_name', 'last_name'])

#column rename
# Rename the 'full_name' column to 'player_name'
merged_NBA_player_game.rename(columns={'full_name': 'player_name'}, inplace=True)


#merged_NBA_player_game.to_csv('23_24_merged_NBA_dataset.csv')

In [10]:
# date formatting to make sure the merge works        NBA DATA
merged_NBA_player_game['GAME_DATE'] = pd.to_datetime(merged_NBA_player_game['GAME_DATE'], format='%b %d, %Y')
merged_NBA_player_game.head(2)

merged_NBA_player_game.to_csv('23_24_merged_NBA_dataset.csv')

In [11]:
updated_merged_player_points_eventid['GAME_DATE'] = pd.to_datetime(updated_merged_player_points_eventid['commence_time']).dt.date
updated_merged_player_points_eventid.head(2)


,Unnamed: 0,id,sport_key,sport_title,commence_time,home_team,away_team,bookmaker_key,bookmaker_title,prop_market,outcome,player_name,price,prop,MATCHUP,GAME_DATE
0,0,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Over,Al Horford,2.35,3.5,BOS vs. PHI,2023-05-04
1,1,f68297ad59b106aaf5970a0a385f539f,basketball_nba,NBA,2023-05-04T00:00:00Z,Boston Celtics,Philadelphia 76ers,draftkings,DraftKings,player_assists,Under,Al Horford,1.61,3.5,BOS vs. PHI,2023-05-04


In [12]:
updated_merged_player_points_eventid.to_csv('eventid_playerprop_pts_rebs_astsV2.csv')

: 

**CREATE a function that uses the merged_NBA_player_game dataframe to determine of the bets hit/miss [1:0] binary outcome**

In [ ]:
#read in updated_merged_player_points_eventid & merged_NBA_player_game



In [1]:
#merge section

# Ensure both 'GAME_DATE' columns are converted to datetime type
updated_merged_player_points_eventid['GAME_DATE'] = pd.to_datetime(updated_merged_player_points_eventid['commence_time']).dt.date
merged_NBA_player_game['GAME_DATE'] = pd.to_datetime(merged_NBA_player_game['GAME_DATE'], format='%b %d, %Y').dt.date

# Merge the two dataframes on 'MATCHUP', 'player_name', and 'GAME_DATE'
final_merged_df = pd.merge(
    updated_merged_player_points_eventid,
    merged_NBA_player_game,
    on=['MATCHUP', 'player_name', 'GAME_DATE'],
    how='left'
)

NameError: name 'pd' is not defined